<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Chat Completion for Decisions</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.5"
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.5 Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> 📜 "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

// Display the result of the chat completion request as Markdown
chatResult.Content.DisplayAs("text/markdown");

Here are five widely recognized decision frameworks that can help improve the quality of decisions:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**  
Developed by military strategist John Boyd, the OODA Loop is a cyclical framework for making decisions in dynamic and fast-changing environments.  
- **Steps**:  
  - **Observe**: Gather information about the situation.  
  - **Orient**: Analyze the information and contextualize it based on prior knowledge and experience.  
  - **Decide**: Choose the best course of action.  
  - **Act**: Implement the decision and monitor its outcomes.  
- **Use Case**: Effective for situations requiring rapid responses, such as business strategy or crisis management.

---

### 2. **The Eisenhower Matrix (Urgent vs. Important)**  
This framework helps prioritize tasks and decisions based on urgency and importance.  
- **Steps**:  
  - Divide tasks into four quadrants:  
    - **Urgent & Important**: Do immediately.  
    - **Important but Not Urgent**: Schedule for later.  
    - **Urgent but Not Important**: Delegate.  
    - **Neither Urgent nor Important**: Eliminate.  
- **Use Case**: Ideal for time management and prioritizing decisions in personal or professional contexts.

---

### 3. **The DECIDE Model**  
A structured approach to decision-making that emphasizes clarity and evaluation.  
- **Steps**:  
  - **D**efine the problem.  
  - **E**stablish criteria for a successful outcome.  
  - **C**onsider all possible options.  
  - **I**dentify the best option.  
  - **D**evelop and implement a plan.  
  - **E**valuate the decision and its outcomes.  
- **Use Case**: Useful for systematic problem-solving and ensuring decisions align with goals.

---

### 4. **Cost-Benefit Analysis (CBA)**  
A quantitative framework for evaluating the pros and cons of different options.  
- **Steps**:  
  - Identify all costs and benefits associated with each option.  
  - Assign monetary or qualitative values to each cost and benefit.  
  - Compare the net benefits of each option to determine the best choice.  
- **Use Case**: Effective for financial decisions, resource allocation, or project planning.

---

### 5. **Six Thinking Hats (Edward de Bono)**  
A creative framework for exploring decisions from multiple perspectives.  
- **Steps**:  
  - Use six metaphorical "hats" to analyze the decision:  
    - **White Hat**: Focus on facts and data.  
    - **Red Hat**: Consider emotions and intuition.  
    - **Black Hat**: Identify risks and challenges.  
    - **Yellow Hat**: Highlight benefits and optimism.  
    - **Green Hat**: Explore creative solutions.  
    - **Blue Hat**: Manage the decision-making process.  
- **Use Case**: Ideal for group decision-making and brainstorming.

---

These frameworks can be adapted to suit various contexts, from personal decisions to complex organizational challenges.

### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt, openAIPromptExecutionSettings))
{
    Console.Write(content);
}

Here are five widely used decision frameworks that can help improve the quality of decisions by providing structured approaches to problem-solving and evaluation:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**  
Developed by military strategist John Boyd, the OODA Loop is a cyclical framework for making decisions in dynamic and fast-changing environments.  
- **Steps**:  
  - **Observe**: Gather information about the situation.  
  - **Orient**: Analyze the information and contextualize it based on prior knowledge and current circumstances.  
  - **Decide**: Choose the best course of action.  
  - **Act**: Implement the decision and monitor its outcomes.  
- **Use Case**: Ideal for situations requiring rapid responses, such as business strategy, crisis management, or competitive environments.

---

### 2. **The Eisenhower Matrix (Urgent vs. Important)**  
This framework helps prioritize tasks and decisions based on urgency and importance.  
- **Steps**:  
  - Divide task

### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

// Note: The chat completion service can also be used with a chat history object, which allows for a more conversational context.
var chatResponse = string.Empty;

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

Here are five structured Decision Frameworks that can improve the quality of decision-making across various contexts:

---

### 1. **Cost-Benefit Analysis Framework**
   - **Overview**: Evaluates the benefits versus the costs associated with a decision to ensure the most advantageous option is chosen.
   - **Steps**:
     1. Identify all possible options.
     2. List potential benefits and costs (financial, time, effort, opportunity cost, and intangible impacts).
     3. Quantify benefits and costs when possible.
     4. Compare net outcomes of each option.
     5. Select the option with the highest net positive result.
   - **Use Cases**: Business investments, resource allocation, personal financial decisions.
   - **Advantages**: Encourages rational thinking and prevents emotional bias.
   - **Pitfalls**: Hard-to-quantify intangible costs/benefits might be ignored.

---

### 2. **The Eisenhower Matrix (Urgent-Important Matrix)**
   - **Overview**: Prioritizes tasks and decisions based on urgency and importance, focusing on what truly matters without overreacting to low-priority demands.
   - **Steps**:
     1. Categorize decisions or tasks into four quadrants:
        - **Quadrant 1**: Urgent and Important (Do it now).
        - **Quadrant 2**: Important but Not Urgent (Schedule for later).
        - **Quadrant 3**: Urgent but Not Important (Delegate).
        - **Quadrant 4**: Neither Urgent nor Important (Eliminate).
     2. Execute tasks based on prioritization.
   - **Use Cases**: Time management, workload prioritization, avoiding procrastination.
   - **Advantages**: Helps identify high-value activities and reduce distractions.
   - **Pitfalls**: Might oversimplify complex decisions.

---

### 3. **Pareto Analysis (80/20 Rule)**
   - **Overview**: Focuses on the vital few actions or decisions that create the most impact (80% of effects come from 20% of causes).
   - **Steps**:
     1. List all possible actions/options for solving a problem or achieving a goal.
     2. Assess the impact of each option on the desired outcome.
     3. Identify the top 20% of actions that contribute 80% of value.
     4. Prioritize resources or efforts on those high-impact items.
   - **Use Cases**: Business efficiency, problem-solving, workflow optimization.
   - **Advantages**: Quickly identifies high-impact decisions/actions.
   - **Pitfalls**: Not every scenario fits into a strict 80/20 distribution.

---

### 4. **Decision Tree Analysis**
   - **Overview**: A visual, hierarchical model that maps out various courses of action and their potential outcomes, including risks and rewards.
   - **Steps**:
     1. Identify the decision you need to make.
     2. Define branches for all possible actions.
     3. Identify the probabilities of outcomes for each action.
     4. Assign values (costs, benefits, or risks) to each path.
     5. Evaluate the expected value of each branch based on probability-weighted outcomes.
     6. Choose the option with the most favorable expected value.
   - **Use Cases**: Risk management, strategic planning, complex decision-making.
   - **Advantages**: Provides a clear visualization for decision paths and possible consequences.
   - **Pitfalls**: Can grow overly complex with too many factors.

---

### 5. **Six Thinking Hats Framework (Edward de Bono)**
   - **Overview**: Encourages considering a problem from multiple perspectives to uncover biases, creativity, risks, and facts.
   - **Steps**:
     1. Wear each "thinking hat" sequentially:
        - **White Hat**: Focus on facts and data.
        - **Red Hat**: Explore emotions and intuitions.
        - **Black Hat**: Analyze risks and challenges (caution).
        - **Yellow Hat**: Focus on benefits and opportunities (optimism).
        - **Green Hat**: Think creatively and outside the box.
        - **Blue Hat**: Manage the process and overall perspective.
     2. Synthesize insights from all perspectives.
     3. Make a balanced and comprehensive decision.
   - **Use Cases**: Brainstorming, group decision-making, innovation.
   - **Advantages**: Prevents tunnel vision and encourages holistic thinking.
   - **Pitfalls**: May require facilitator-led practice to master.

---

### Final Thoughts
Each of these frameworks is suitable depending on the decision context. For quick prioritization of tasks, the Eisenhower Matrix works well; for weighing pros and cons, the Cost-Benefit Analysis is ideal; for a creative approach to brainstorming, Six Thinking Hats can encourage holistic thinking. Combining decision frameworks in more complex scenarios may also enhance results systematically.

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> 📜 "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community?  
Think carefully step by step about what decision frameworks are needed to answer the query.  
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

To determine which decision framework is best suited for the military intelligence community, it’s important to analyze the characteristics of the environment, the expectations of intelligence work, and the qualities of effective decision-making within such contexts. Let’s evaluate step-by-step which framework is most appropriate.

---

### Step 1: **Understand the Military Intelligence Context**
Military intelligence operates in environments that are:
- **Dynamic:** Situations can change quickly, often unpredictably.
- **Complex:** Decisions involve analyzing large amounts of data, understanding adversary behavior, and assessing risks to national security.
- **Uncertain:** There is often incomplete or ambiguous information available.
- **Time-sensitive:** Delays can have significant operational consequences and may undermine mission success.
- **Iterative:** Continuous analysis and reevaluation of situations are critical as new information becomes available.

The decision-making framework will likely need to address **uncertainty**, **agility**, and **iteration**, while minimizing biases and maximizing operational efficiency.

---

### Step 2: **Evaluate the Five Frameworks**
Let’s examine each framework against the criteria established above:

#### 1. **Cynefin Framework**
The Cynefin Framework recognizes the complexity and dynamic nature of environments by categorizing them into simple, complicated, complex, and chaotic domains. Given the diverse nature of military intelligence situations (e.g., strategy formulation, crisis management, and battlefield tactics), this framework provides a high-level structure for understanding the type of decision-making required. However, it lacks specific procedural guidance for real-time, iterative intelligence analysis.

#### 2. **Six Thinking Hats**
While the Six Thinking Hats framework encourages diverse perspectives and balances emotions, creativity, and logic, it’s better suited to group deliberations and brainstorming sessions. The intelligence community requires decision-making frameworks that emphasize urgency, agility, and iterative processes—which are less emphasized in the Six Thinking Hats approach.

#### 3. **OODA Loop**
The OODA Loop (Observe, Orient, Decide, Act) is specifically designed for dynamic, fast-changing environments that require iterative decisions based on evolving information. Intelligence gathering aligns naturally with the **Observe** phase, where data is collected and analyzed. The **Orient** phase allows analysts to contextualize data (e.g., geopolitical factors, adversary motivations). The **Decide** phase enables quick course selection, and the **Act** phase ensures execution of a decision while preparing for the next iteration. This framework is highly effective for military intelligence because it accounts for uncertainty, time sensitivity, and adaptation to new information.

#### 4. **Decision Matrix Analysis**
The Decision Matrix is ideal for structured, analytical reasoning with multiple criteria. However, military intelligence often involves fluid and uncertain situations that demand adaptation rather than static comparisons. While useful for certain technical evaluations (e.g., selecting equipment, planning logistics), the matrix lacks the agility needed to navigate the fast-paced nature of intelligence work.

#### 5. **Eisenhower Matrix**
The Eisenhower Matrix is designed for prioritizing tasks (urgent vs. important) rather than making comprehensive operational decisions. Though prioritization is necessary for intelligence, military operations often deal with overlapping urgency and importance, making this framework less suitable for navigating complex, evolving circumstances.

---

### Step 3: **Reasoning Through Selection**
The framework most suited for the military intelligence community must:
- Enable real-time adaptation to new data and circumstances.
- Function effectively under uncertainty and time pressures.
- Support iterative decision-making as intelligence evolves.

Among the frameworks, the **OODA Loop** aligns the most closely with these requirements. Its focus on agility, iterative thinking, and rapid response makes it ideal for military intelligence scenarios.

---

### Final Recommendation: **OODA Loop**

The OODA Loop (Observe, Orient, Decide, Act) is the single best framework for use within the military intelligence community. This framework supports critical real-time situational awareness and decision-making in dynamic, high-stakes environments, enabling intelligence professionals to adapt to evolving challenges while maintaining operational effectiveness.

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> 📜 "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

Semantic Kernel can maintain state with the ChatHistory object. This allows natural converation between the user and AI to be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [14]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
// Notice how the conversation flow uses different roles (user and assistant) to simulate a realistic decision-making dialogue
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation history as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. This "Gathered Intelligence" for the decision can be persisted, re-loaded, simulated multiple times, applied to different AI systems to optimize decisions further.  

In [15]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistoryDecisionConversation.AddAssistantMessage(chatResponse);

The Fitbit Inspire 3 is an excellent choice! It offers up to 10 days of battery life, reliable sleep tracking (including detailed sleep stages), and an intuitive app for health insights—all at an affordable price point. Plus, it’s lightweight and comfortable for 24/7 wear.

**Final considerations:**
1. **Compatibility:** Ensure it syncs well with your smartphone (works with both iOS and Android).
2. **Extras:** Explore Fitbit Premium for extra insights (often comes with a trial period).
3. **Purpose:** Whether you're tracking steps, sleep, or general wellness, revisit how it aligns with your primary goals as you set it up.

You seem to have made a well-reasoned decision! Enjoy tracking and optimizing your health journey. 😊

### Step 6 - Inspect & Optimize Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

> 📝 Note: In the Decision Intelligence framework, chat history can serve as a form of gathered intelligence. Interactions among users, AI models, processes, and agents provide valuable context for effective decision-making. It’s highly recommended to persist chat history objects (agent threads), especially during decision optimization, to ensure critical information is retained and accessible.  

In [16]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

// Change this to a string builder and show as markdown
var stringBuilderChatHistory = new StringBuilder();
foreach (var message in chatHistoryDecisionConversation)
{
    // add a new line for each message
    stringBuilderChatHistory.AppendLine($"**{message.Role.ToString().ToUpper()}**:");
    stringBuilderChatHistory.Append($"{message.Content.Replace("#", string.Empty)}");
    stringBuilderChatHistory.AppendLine("\n");
}

// Display the chat history as Markdown
stringBuilderChatHistory.ToString().DisplayAs("text/markdown");

Number of chat interactions: 11


**SYSTEM**:
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

**USER**:
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

**ASSISTANT**:
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

**USER**:
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?

**ASSISTANT**:
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.

**USER**:
Are there any popular models you would recommend for that balance of battery life and sleep tracking?

**ASSISTANT**:
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.

**USER**:
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?

**ASSISTANT**:
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 

**USER**:
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts on my decision? 

**ASSISTANT**:
The Fitbit Inspire 3 is an excellent choice! It offers up to 10 days of battery life, reliable sleep tracking (including detailed sleep stages), and an intuitive app for health insights—all at an affordable price point. Plus, it’s lightweight and comfortable for 24/7 wear.

**Final considerations:**
1. **Compatibility:** Ensure it syncs well with your smartphone (works with both iOS and Android).
2. **Extras:** Explore Fitbit Premium for extra insights (often comes with a trial period).
3. **Purpose:** Whether you're tracking steps, sleep, or general wellness, revisit how it aligns with your primary goals as you set it up.

You seem to have made a well-reasoned decision! Enjoy tracking and optimizing your health journey. 😊



By persisting the Decision Intelligence chats and making them available for inspection, can further help the decision-making process. For example you can load the Chat History summarize it or even have AI recommend optimizations for it based on the conversation. 

> 📝 Note: The process below is simplified to illustrate the power of Gathering Intelligence with user <--> AI interactions. More advanced best practices will be introduced in further sections. 

In [17]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
chatResponse = chatContent.Content;
chatResponse.DisplayAs("text/markdown");

// Optimize the chat history for any future decision interactions
var userPromptOptimizeTheDecisionChat = """
Based on the chat conversation so far...
Provide some recommendations on how the decision interactions could be optimized for decision-making.
What are some considerations for the AI to help the use make even better decisions?
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptOptimizeTheDecisionChat);

var chatContentOptimization = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
var chatResponseOptimization = chatContentOptimization.Content;
chatResponseOptimization.DisplayAs("text/markdown");

Here’s a summary of our conversation:

You expressed interest in buying a fitness tracker and sought advice on key considerations. We discussed features like fitness tracking, health monitoring, battery life, sleep tracking, comfort, and price. You prioritized battery life and sleep tracking, and I shared typical battery life ranges (2-14 days) and the capabilities of modern sleep tracking tools.

We reviewed specific models like the Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, Xiaomi Mi Band 7, and the Apple Watch. While the Apple Watch offers advanced features, its shorter battery life (18-24 hours) made it less suitable for your needs.

Ultimately, you chose the Fitbit Inspire 3 due to its affordability, long battery life, and robust sleep tracking. We briefly discussed compatibility and Fitbit Premium for added value, concluding that your decision aligns well with your priorities.

**Summary of the Chat Conversation:**
The user explored purchasing a fitness tracker with a focus on battery life and sleep tracking features. Key considerations such as fitness features, health monitoring, and compatibility were discussed. Popular models like Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, and Xiaomi Mi Band 7 were recommended. Apple Watch was evaluated but ruled out due to shorter battery life, despite its robust features. The user decided on the Fitbit Inspire 3 after weighing the pros and cons of various options.

---

**Recommendations for Optimizing Decision Interactions:**

1. **Structured Decision Frameworks:**
   - Introduce a decision-making framework, such as pros and cons, weighted criteria, or ranking systems, early in the conversation.
   - For example: "Rank these features (battery life, sleep tracking, price, design, etc.) on a scale of importance to better identify the best option."

2. **Encourage Exploration of Scenarios:**
   - Suggest scenarios for how the user will use the tracker and then match recommendations to those needs (e.g., daily activity tracking vs. specialized metrics for athletes vs. health monitoring for sleep optimization).

3. **Cost-Benefit Analysis:**
   - Prompt the user to compare/personalize price to value, ensuring the decision stays within their preferred budget and feature priorities while acknowledging tradeoffs.

4. **Highlight Hidden Features or Considerations:**
   - Suggest lesser-considered aspects, such as ease of app use, durability, third-party integrations, or whether features like community challenges or reminders encourage long-term engagement.

5. **Decision Validation or Next Steps:**
   - Offer actionable post-decision guidance (e.g., setup tips, how to test the device’s effectiveness for their goals, alternative options if the chosen tracker doesn’t perform as expected).

6. **Interactive Tools or Visuals:**
   - If possible, supply comparison charts, feature matrices, or links to relevant reviews to help the user make a more informed and confident decision.

7. **Manage Long-Term Implications:**
   - Encourage foresight by discussing upgrade paths, warranties, and future compatibility considerations (e.g., new fitness features or smartwatch integration).

By implementing structured guidance, enriching feature discussions, and validating decisions with follow-up next steps, the AI can enhance decision-making interactions and foster greater confidence in the user's choice.